# Keras tutorial

In [ ]:
! git clone https://github.com/btlgs2000/dl_intro

In [ ]:
import random

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
import numpy as np

from dl_intro.utils import display_samples, take_test_samples_idxs

## MNIST Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
x_train[0].shape

In [ ]:
display_samples(x_train, y_train, n_rows=5, n_cols=8, label=6)

## MLP con modello sequenziale

### lista di strati passati al costruttore

In [ ]:
model = keras.Sequential([
    Input(shape=(28, 28)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
]
)

In [ ]:
model.summary()

### Costruzione con il metodo add

In [ ]:
model = keras.Sequential()
model.add(Input(shape=(28, 28)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

### sommario

In [ ]:
model.summary()

### plot

In [ ]:
keras.utils.plot_model(model)

### compilazione

La compilazione prepara il modello per l'addestramento.
Vengono definiti:
* il metodo di ottimizzazione
* la funzione di loss
* eventuali metriche

In [ ]:
sgd = keras.optimizers.SGD(learning_rate=1e-3)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

### addestramento

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='tensorboard_logs', histogram_freq=1, write_graph=True, write_images=True),
    tf.keras.callbacks.ModelCheckpoint('tensorboard_logs/weights.{epoch:02d}-{val_loss:.2f}.tf', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [ ]:
h = model.fit(x=x_train, y=tf.one_hot(y_train, 10), batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

### test

In [ ]:
model.metrics_names

In [ ]:
val_loss, val_accuracy = model.evaluate(x=x_test, y=tf.one_hot(y_test, 10), batch_size=32)

### predizione

In [ ]:
y_prob = model.predict(x=x_test)

In [ ]:
y_hat = np.argmax(y_prob, axis=1)

## esame della predizione

### predizioni corrette con maggiore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=True, most_confident=True, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni corrette con minore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=True, most_confident=False, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni errate con maggiore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=False, most_confident=True, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)

### predizioni errate con minore confidenza

In [ ]:
idxs = take_test_samples_idxs(y_prob, y_test, is_correct=False, most_confident=False, k=5*4)
display_samples(x_test[idxs], y_test[idxs], y_hat[idxs], y_prob[idxs], n_rows=5, n_cols=4)